## 정보보호와 시스템보안
### Team Project #1

### 팀 정보
- 팀 이름 : 
- 팀 번호 :10
- 팀원 목록 : 이한정 (20191650), 김지홍 (20191572)

---

### 프로젝트 TODO
1. Data Preprocessing
2. Data Scaling
3. IP Voting

### 프로젝트 데이터 셋
- 국민대학교 네트워크 플로우 데이터셋(5시간)   

### 참고자료
- 네트워크 플로우([Network Flow](https://checkmk.com/guides/network-flow-monitoring)):   
한번의 세션에서 출발지(Source IP)와 도착지(Destination IP) 사이에 오가는 패킷에 대한 통계적 데이터
### 주의사항
- 기존에 있는 파일과 폴더의 구조를 변경할 수 없음

### 제출
- `<팀 번호>_Project1.ipynb`파일을 팀장이 제출   
팀 번호 : 구글 스프레드 시트 행 인덱스

In [1]:
from google.colab import drive

drive_path = '/content/drive'
drive.mount(drive_path)

Mounted at /content/drive


In [2]:
!7z x /content/drive/MyDrive/2022_security_project1.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/                                 1 file, 178076816 bytes (170 MiB)

Extracting archive: /content/drive/MyDrive/2022_security_project1.zip
--
Path = /content/drive/MyDrive/2022_security_project1.zip
Type = zip
Physical Size = 178076816

  0%    
Enter password (will not be echoed):
  0% 1 - Security Project1/BCE_model.py                                         2% 5 - Security Project1/problem1_answer/answer_train_list.pkl                                                                

In [136]:
%cd "./Security Project1"

[Errno 2] No such file or directory: './Security Project1'
/content/Security Project1


In [137]:
from Utils import open_pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

### train, valid 데이터 셋 불러오기
- 가상대학을 통해 다운받은 플로우 데이터를 불러온다.

In [193]:
train_path = r"./train.csv"
valid_path = r"./valid.csv"

train_df = pd.read_csv(train_path)
valid_df = pd.read_csv(valid_path)

In [194]:
# 학습, 검증 플로우 데이터 개수 확인
print(len(train_df), len(valid_df))

977270 325560


In [195]:
# 주어진 플로우 컬럼 종류 확인
train_df.keys()

Index(['Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol', 'Flags',
       'Timestamp', 'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Std', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd Header Length', 'Bwd Header Length',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Std',
       'Packet Length Variance', 'Average Packet Size', 'Fwd Segment Size Avg',
       'Bwd Segment Size Avg', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max'

In [196]:
flags_dic = {'C':2, 'E':3, 'U':5, 'A':7, 'P':11, 'R':13, 'S':17, 'F':19}
flags = ['........', '......S.', '...A..S.', '...A.RS.', 'CE.APRS.',
       '...AP...', '...A...F', '...APR..', '...AP..F', '...A..SF',
       '.....RS.', '...A.RSF', '...APRS.', '...A.R..', '...AP.SF',
       'CE....S.', '...A....', 'CE.AP.S.', '...AP.S.', '..U...S.',
       'CE.A.RS.', 'CE.AP.SF', '...A.R.F', '...APR.F', 'CE.A.RSF',
       'CE.A..SF', '.....R..', '.E.A..S.', 'CE.A..S.', '...APRSF'] # 테스트에서 밑에 unique 결과에서 처음본 애 있으면 추가해줘야함
encoded_flags = []

for flag in tqdm(flags):
    flag = flag.replace('.','')
    encoded_flag = 1
    for s in flag:
        encoded_flag *= flags_dic[s]
    encoded_flags.append(encoded_flag)
    
for idx, flag in enumerate(flags):
    train_df.loc[train_df['Flags'] == flag, 'Flags'] = encoded_flags[idx]
    
for idx, flag in enumerate(flags):
    valid_df.loc[valid_df['Flags'] == flag, 'Flags'] = encoded_flags[idx]

100%|██████████| 30/30 [00:00<00:00, 24808.58it/s]


In [197]:
valid_df['Flags'].unique()

array([133, 17, 1, 7, 119, 102, 1547, 221, 24871, 91, 77, 1309, 1463,
       19019, 85, 2261, 29393, 323323, 7854, 102102, 1001, 9282, 13,
       149226, 17017, 714], dtype=object)

In [198]:
protocols = ['TCP', 'UDP', '0']
encoded_protocols = [1, -1, 0]

for idx, protocol in enumerate(protocols):
    train_df.loc[train_df['Protocol'] == protocol, 'Protocol'] = encoded_protocols[idx]
    
for idx, protocol in enumerate(protocols):
    valid_df.loc[valid_df['Protocol'] == protocol, 'Protocol'] = encoded_protocols[idx]

In [199]:
# train_df = train_df.sort_values('Timestamp')
# train_df.head(20)

In [200]:
# 3분마다 그룹핑
# time_group = {i : [] for i in range(3, 24*60+4, 3)}
# for row in train_df.itertuples():
#   times = list(map(int, row.Timestamp.split(':')))
#   time = times[0]*60 + times[1] # 분 단위만 생각
#   time_key = (time//3 + 1) * 3
#   time_group[time_key].append(row)

In [201]:
# print(time_group[3][0])

In [202]:
train_df = train_df.rename(columns={'Src Port': 'src_port', 'Dst Port': 'dst_port', 'Src IP': 'src_ip', 'Dst IP': 'dst_ip'})
valid_df = valid_df.rename(columns={'Src Port': 'src_port','Dst Port': 'dst_port', 'Src IP': 'src_ip', 'Dst IP': 'dst_ip'})

In [203]:
train_df = train_df.astype({'src_port': 'float', 'dst_port': 'float'})
valid_df = valid_df.astype({'src_port': 'float', 'dst_port': 'float'})

In [204]:
# src_port_count = dict(train_df['src_port'].value_counts())
# dst_port_count = dict(train_df['dst_port'].value_counts())
# num_src_port = float(train_df['src_port'].count())
# num_dst_port = float(train_df['dst_port'].count())

In [205]:
# port CDF 인코딩 테이블 만들기
# for key, value in src_port_count.items():
#   src_port_count[key] = value/num_src_port

# for key, value in dst_port_count.items():
#   dst_port_count[key] = value/num_dst_port

In [206]:
# sorted_src_port_table = dict(sorted(src_port_count.items(), key=lambda x: x[1]))
# prev_value = list(sorted_src_port_table.values())[0]
# prev_idx = 0
# cum_sum = 0
# cum_key = []

# for key, value in sorted_src_port_table.items():
#   if value != prev_value:
#     for k in cum_key:
#         sorted_src_port_table[k] = cum_sum
#     cum_key = [key]
#     cum_sum += value
#     prev_value = value
#   else:
#     cum_key.append(key)
#     cum_sum += value
#     prev_value = value
    
# if cum_key:
#     for k in cum_key:
#         sorted_src_port_table[k] = cum_sum
# sorted_src_port_table

In [207]:
# sorted_dst_port_table = dict(sorted(dst_port_count.items(), key=lambda x: x[1]))
# prev_value = list(sorted_dst_port_table.values())[0]
# prev_idx = 0
# cum_sum = 0
# cum_key = []

# for key, value in sorted_dst_port_table.items():
#   if value != prev_value:
#     for k in cum_key:
#         sorted_dst_port_table[k] = cum_sum
#     cum_key = [key]
#     cum_sum += value
#     prev_value = value
#   else:
#     cum_key.append(key)
#     cum_sum += value
#     prev_value = value
    
# if cum_key:
#     for k in cum_key:
#         sorted_dst_port_table[k] = cum_sum
# sorted_dst_port_table.values()

In [208]:
# for key, value in sorted_src_port_table.items():
#     train_df.loc[train_df['src_port'] == key, 'src_port'] = value
# for key, value in sorted_dst_port_table.items():
#     train_df.loc[train_df['dst_port'] == key, 'dst_port'] = value
# for key, value in sorted_src_port_table.items():
#     valid_df.loc[valid_df['src_port'] == key, 'src_port'] = value
# for key, value in sorted_dst_port_table.items():
#     valid_df.loc[valid_df['dst_port'] == key, 'dst_port'] = value
# # train_df['src_port'].replace(sorted_src_port_table, inplace=True)
# # train_df['dst_port'].replace(sorted_dst_port_table, inplace=True)
# # valid_df['src_port'].replace(sorted_src_port_table, inplace=True)
# # valid_df['dst_port'].replace(sorted_dst_port_table, inplace=True)

# valid_df.loc[valid_df['src_port'] >= 2, 'src_port'] = 0 # train에 등장 안해서 테이블에 없는애들 0으로 변경
# valid_df.loc[valid_df['dst_port'] >= 2, 'dst_port'] = 0

In [209]:
train_df.head(5)

,src_ip,src_port,dst_ip,dst_port,Protocol,Flags,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,110.15.155.169,13850.0,97.2.159.1,53.0,1,17,03:52:00,3119967,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
1,110.15.155.169,43842.0,97.2.159.1,53.0,1,17,03:37:20,2991830,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
2,110.15.155.171,25918.0,97.2.159.1,53.0,1,17,03:10:28,3004658,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
3,238.61.97.110,4445.0,97.2.159.1,53.0,1,133,00:42:24,4402645,2,1,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
4,110.15.155.171,28467.0,97.2.159.1,53.0,1,17,02:45:28,3001784,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP


In [210]:
valid_df.head(5)

,src_ip,src_port,dst_ip,dst_port,Protocol,Flags,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,238.61.97.10,61650.0,97.2.159.1,53.0,1,133,00:47:49,4454322,2,1,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
1,110.15.155.171,35929.0,97.2.159.1,53.0,1,17,04:05:32,2988477,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
2,238.61.97.10,64652.0,97.2.159.1,53.0,1,133,02:07:05,5092931,2,1,...,20,0.0,0.0,0.0,0.0,5090263.0,0.0,5090263.0,5090263.0,Benign_IP
3,110.15.155.169,42876.0,97.2.159.1,53.0,1,17,03:19:22,2993254,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
4,238.61.97.10,62593.0,97.2.159.1,53.0,1,133,00:53:13,4534770,2,1,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP


## Problem #1 : Data Preprocessing

### 조건
- "Label"컬럼을 사용하여, 악성 IP의 플로우는 1, 정상 IP의 플로우는 0으로 구성된 라벨 리스트 생성   
=> `train_y`, `valid_y`(변수명 유지)
- 예외처리 컬럼 "Src IP", "Dst IP", "Protocol", "Timestamp", "Flags", "Label"을 제외,   
8개 컬럼을 제외한 48컬럼을 사용하여 데이터셋 생성   
- 4개의 `list` 모두 csv 파일의 플로우 순서 유지
- hint : 결과 파일을 분석하여 코드를 구상한다.

### 관련 정보
- [pandas](https://pandas.pydata.org/docs/) library, [csv](https://docs.python.org/ko/3/library/csv.html) module
- 플로우 피처 정보(주어진 ppt 특정 부분으로 명시)

### I/O
- input :
    > - `train.csv`
    > - `valid.csv`
- output :   
    > - `train_y` -> `list`
    > - `valid_y` -> `list`
    > - `train_list` -> `list`
    > - `valid_list` -> `list`

In [211]:
#TODO1
from tqdm import tqdm

train_y = []
valid_y = []

for row in tqdm(train_df.itertuples()):
  if row.Label == 'Benign_IP':
    train_y.append(0)
  else:
    train_y.append(1)
  
for row in tqdm(valid_df.itertuples()):
  if row.Label == 'Benign_IP':
    valid_y.append(0)
  else:
    valid_y.append(1)

dropped_train_df = train_df.drop(["src_ip", "dst_ip", "Flags", "Protocol", "Timestamp", "Label"], axis=1)
dropped_valid_df = valid_df.drop(["src_ip", "dst_ip", "Flags", "Protocol", "Timestamp", "Label"], axis=1)

train_list = dropped_train_df.values.tolist()
valid_list = dropped_valid_df.values.tolist()

977270it [00:08, 112607.59it/s]
325560it [00:04, 79891.25it/s] 


## Problem #2 : Data Scaling

### 조건
- sklearn의 [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)사용
- scaler는 학습 데이터로만 완성해야한다.

### 관련 정보
[numpy](https://numpy.org/devdocs/) 패키지

## I/O
- input :
    > - `train_list`
    > - `valid_list`
- output :   
    > - `train_X` -> `ndarray(float64)`
    > - `valid_X` -> `ndarray(float64)`

In [212]:
from sklearn.preprocessing import StandardScaler 

#TODO2 - StandardScaler
scaler = StandardScaler()
scaler.fit(train_list)
train_X = scaler.transform(train_list)
valid_X = scaler.transform(valid_list)
# 이거 스케일러 하고 새로만든애들 뒤에 붙이는거 해야함

In [213]:
train_flags = list(train_df['Flags'])
train_protocols = list(train_df['Protocol'])
train_src_port = list(train_df['src_port'])
train_dst_port = list(train_df['dst_port'])

valid_flags = list(valid_df['Flags'])
valid_protocols = list(valid_df['Protocol'])
valid_src_port = list(valid_df['src_port'])
valid_dst_port = list(valid_df['dst_port'])

In [214]:
print(len(train_X[0]))
print(len(valid_X[0]))

48
48


In [215]:
train_X = train_X.tolist()
valid_X = valid_X.tolist()

for i in range(len(train_src_port)):
    train_X[i].extend([train_flags[i], train_protocols[i]])
# train_flags[i], train_protocols[i], train_src_port[i], train_dst_port[i]]
for i in range(len(valid_src_port)):
    valid_X[i].extend([valid_flags[i], valid_protocols[i]])
# valid_flags[i], valid_protocols[i], valid_src_port[i], valid_dst_port[i]   

In [216]:
print(len(train_X[0])) # 50이 맞음

50


In [217]:
print(len(valid_X[0]))

50


In [218]:
train_X = np.array(train_X)
valid_X = np.array(valid_X)
print(train_X[0])
print(valid_X[0])

[-1.68449750e+00 -6.76570579e-01 -3.13958122e-01 -1.27507466e-01
 -2.85610333e-02 -2.16662096e-02 -1.51682014e-02 -2.46803484e-01
 -2.03180650e-01 -1.88251439e-01 -2.47357845e-01 -2.26019375e-01
 -1.84897962e-01 -9.56507268e-02 -2.09577864e-01 -2.41336547e-01
  8.95805494e-02 -3.13650711e-01 -1.02502364e-01 -2.11370797e-01
 -2.41384938e-01  3.62777834e-03 -9.60672262e-02 -6.91518973e-02
 -4.16791024e-02 -7.52894036e-02 -6.50352748e-02 -2.61164807e-01
 -2.98831647e-02 -1.95112205e-01 -2.71795178e-01 -2.52927089e-01
 -1.64156663e-01 -3.06353392e-01 -2.72818610e-01 -2.89858154e-01
  4.55244321e-01 -1.91775787e-01 -3.79851294e-02  1.56933206e+00
 -3.72437940e-01 -7.91748085e-02 -3.37192686e-01 -3.72378677e-01
 -3.84330641e-01 -3.05902963e-01 -3.99856416e-01 -3.19993488e-01
  1.70000000e+01  1.00000000e+00]
[ 9.41768016e-01 -6.76570579e-01 -2.60998565e-01 -1.30445310e-01
 -2.21750403e-02 -2.16662096e-02 -1.51682014e-02 -2.46803484e-01
 -2.03180650e-01 -1.88251439e-01 -2.47357845e-01 -2.2601

In [219]:
#학습을 위한 데이터 타입 변환
from BCE_model import train, test

train_X = train_X.astype(np.float32)
valid_X = valid_X.astype(np.float32)
train_y = np.array(train_y, dtype=np.float32)
valid_y = np.array(valid_y, dtype=np.float32)

In [220]:
# Project1 - train 데이터 학습 및 valid 데이터 예측

# model = train(train_X, train_y)
# predict = test(valid_X, valid_y, model)

In [221]:
from sklearn.ensemble import RandomForestClassifier # F-1 Score: 0.91

model = RandomForestClassifier().fit(train_X, train_y)
predict = model.predict(valid_X)
model.score(valid_X, valid_y)

0.9925942990539378

In [222]:
predict = np.array(predict, dtype="float32")

In [223]:
# 성능 평가를 위해 예측 값을 반올림
round_valid_y = valid_y.astype("int")
round_predict = np.round(predict)

성능 평가 지표 관련 자료 : https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/   
*주의사항 : 이후 진행할 Problem #3의 채점을 위해 학습 결과가 보기 예시와 동일하게 나오는지 확인

In [224]:
# 플로우 기준 이상 탐지 성능 평가
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
print("confusion_matrix\n", confusion_matrix(round_valid_y, round_predict))
print(f"accuracy\t: {accuracy_score(round_valid_y, round_predict): .5f}")
print(f"precision\t: {precision_score(round_valid_y, round_predict): .5f}")
print(f"recall\t\t: {recall_score(round_valid_y, round_predict): .5f}")
print(f"F-1\t\t: {f1_score(round_valid_y, round_predict): .5f}")

confusion_matrix
 [[188579   1694]
 [   717 134570]]
accuracy	:  0.99259
precision	:  0.98757
recall		:  0.99470
F-1		:  0.99112


In [225]:
# 정상 외부 IP set
outer_benignIP_set = open_pickle("./outer_benign_IP.pkl")
# 악성 외부 IP set
outer_malIP_set = open_pickle("./outer_mal_IP.pkl")

In [226]:
#외부 IP의 악성, 정상 개수 확인
len(outer_benignIP_set), len(outer_malIP_set)

(590, 457)

In [227]:
# Problem #3을 위해 dictionary형태로 변경
labelIP_dict = dict()

for ip in outer_benignIP_set:
    labelIP_dict[ip] = 0
for ip in outer_malIP_set:
    labelIP_dict[ip] = 1

In [228]:
len(labelIP_dict)

1047

## Problem #3 : IP Voting
### 조건
- 모든 flow는 내부IP <-> 외부IP 통신으로 이루어져있다.
- 외부IP는 악성IP, 정상IP로 나뉜다.
- 각 외부 IP별로 1개 이상의 플로우를 가진다.
- valid.csv의 각 외부IP별로 플로우를 분류한다.
- 각 외부 IP에서 속한 플로우가 1개라도 `theta`이상의 predict값을 가진다면 악성 외부 IP로 예측한다.
- 외부IP를 key, 예측 값을 value로 가지는 dictionary를 반환한다.(`labelIP_dict`과 같은 형태)
- 예측 라벨은 악성 외부 IP를 `1`, 정상 외부 IP를 `0`으로 한다. 
- 주어진 모든 `theta`에 대해 작동할 수 있도록 한다. (*채점시 다른 `theta`적용 후 결과 비교 예정)
- hint : csv파일의 순서와 predict의 순서는 일치한다.

### I/O
- input :
    > - `valid.csv`
    > - `predict`
    > - `labelIP_dict`
    > - `theta`
- output :   
    > - `predictIP_dict` -> `dict`

In [233]:
theta = 0.1 # example

#TODO3
predictIP_dict = dict()
external_IP =list(labelIP_dict.keys())
external_IP_group = dict()

valid_df = valid_df.rename(columns={'Src IP': 'src_ip'})
valid_df = valid_df.rename(columns={'Dst IP': 'dst_ip'})

# grouping
for row in tqdm(valid_df.itertuples()):
  src_ip = row.src_ip
  dst_ip = row.dst_ip

  if src_ip in external_IP:
    if src_ip in external_IP_group:
      external_IP_group[src_ip].append(row)
    else:
      external_IP_group[src_ip] = [row]
  else:
    if dst_ip in external_IP_group:
      external_IP_group[dst_ip].append(row)
    else:
      external_IP_group[dst_ip] = [row]


# predict
# for key, values in tqdm(external_IP_group.items()):
#   predictIP_dict[key] = 0

#   for value in values:
#     if predict[value.Index] >= theta:
#       predictIP_dict[key] = 1
#       break


# if proportion 0.7, F-1 Score: 0.983
proportion = 0.7
for key, values in tqdm(external_IP_group.items()):
  predictIP_dict[key] = 0
  cnt = 0
  for value in values:
    if predict[value.Index] == 1:
      cnt += 1
      if cnt/float(len(values)) >= proportion:
          predictIP_dict[key] = 1
          break

325560it [00:09, 34670.04it/s]
100%|██████████| 1047/1047 [00:00<00:00, 1273.53it/s]


In [230]:
print(predictIP_dict.values())

dict_values([0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 

In [231]:
import pickle

with open("./answer_predictIP_dict_new.pkl", 'wb') as f:
    pickle.dump(predictIP_dict, f)

In [234]:
# 정답 IP dictionary와 예측 IP dictionary를 비교하여 성능 평가
pred_IP = []
label_IP = []

for k in predictIP_dict.keys():
    pred_IP.append(predictIP_dict[k])
    label_IP.append(labelIP_dict[k])

print("confusion_matrix\n", confusion_matrix(label_IP, pred_IP))
print(f"accuracy\t: {accuracy_score(label_IP, pred_IP): .5f}")
print(f"precision\t: {precision_score(label_IP, pred_IP): .5f}")
print(f"recall\t\t: {recall_score(label_IP, pred_IP): .5f}")
print(f"F-1\t\t: {f1_score(label_IP, pred_IP): .5f}")

confusion_matrix
 [[581   9]
 [  6 451]]
accuracy	:  0.98567
precision	:  0.98043
recall		:  0.98687
F-1		:  0.98364
